In [1]:
import requests
import os
import sys
import pandas as pd
import json

In [3]:
if 'google.colab' in sys.modules:
    root_dir = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'ff14')
else:
    root_dir = os.path.join(os.getcwd(), 'ff14')

In [20]:
ITEM_CATEGORY_CONFIG_MAP = {
    'All': 0,  # All
    'Pugilist Weapons': 1,  # Equipment
    'One-handed Swords': 2,  # Equipment
    'Greataxes': 3,  # Equipment
    'Bows': 4,  # Equipment
    'Polearms': 5,  # Equipment
    'One-handed Conjurer Arms': 6,  # Equipment
    'Two-handed Conjurer Arms': 7,  # Equipment
    'One-handed Thaumaturge Arms': 8,  # Equipment
    'Two-handed Thaumaturge Arms': 9,  # Equipment
    'Grimoires': 10,  # Equipment
    'Shields': 11,  # Equipment
    'Carpenter\'s Primary Tool': 12,  # Equipment
    'Carpenter\'s Secondary Tool': 13,  # Equipment
    'Blacksmith\'s Primary Tool': 14,  # Equipment
    'Blacksmith\'s Secondary Tool': 15,  # Equipment
    'Armorer\'s Primary Tool': 16,  # Equipment
    'Armorer\'s Secondary Tool': 17,  # Equipment
    'Goldsmith\'s Primary Tool': 18,  # Equipment
    'Goldsmith\'s Secondary Tool': 19,  # Equipment
    'Leatherworker\'s Primary Tool': 20,  # Equipment
    'Leatherworker\'s Secondary Tool': 21,  # Equipment
    'Weaver\'s Primary Tool': 22,  # Equipment
    'Weaver\'s Secondary Tool': 23,  # Equipment
    'Alchemist\'s Primary Tool': 24,  # Equipment
    'Alchemist\'s Secondary Tool': 25,  # Equipment
    'Culinarian\'s Primary Tool': 26,  # Equipment
    'Culinarian\'s Secondary Tool': 27,  # Equipment
    'Miner\'s Primary Tool': 28,  # Equipment
    'Miner\'s Secondary Tool': 29,  # Equipment
    'Botanist\'s Primary Tool': 30,  # Equipment
    'Botanist\'s Secondary Tool': 31,  # Equipment
    'Fisher\'s Primary Tool': 32,  # Equipment
    'Fishing Tackle': 33,  # Materials
    'Head Armor': 34,  # Equipment
    'Body Armor': 35,  # Equipment
    'Leg Armor': 36,  # Equipment
    'Hand Armor': 37,  # Equipment
    'Foot Armor': 38,  # Equipment
    'Waist Armor': 39,  # Equipment
    'Necklaces': 40,  # Equipment
    'Earrings': 41,  # Equipment
    'Bracelets': 42,  # Equipment
    'Rings': 43,  # Equipment
    'Potions': 44,  # Consumables
    'Ingredients': 45,  # Materials
    'Food': 46,  # Consumables
    'Fish': 47,  # Materials
    'Minerals': 48,  # Materials
    'Metals': 49,  # Materials
    'Lumber': 50,  # Materials
    'Cloth': 51,  # Materials
    'Leather': 52,  # Materials
    'Bone': 53,  # Materials
    'Alchemical Materials': 54,  # Materials
    'Dyes': 55,  # Dyes
    'Parts': 56,  # Materials
    'Furnishings': 57,  # Furnishings
    'Materia': 58,  # Consumables
    'Crystals': 59,  # Materials
    'Catalysts': 60,  # Materials
    'Sundries': 61,  # Sundries
    'Soul Crystals': 62,  # Soul Crystals
    'Other': 63,  # Other
    'Estate Certificates': 64,  # Estate Certificates
    'Roofs': 65,  # Roofs
    'Exterior Walls': 66,  # Exterior Walls
    'Windows': 67,  # Windows
    'Doors': 68,  # Doors
    'Roof Decorations': 69,  # Roof Decorations
    'Exterior Wall Decorations': 70,  # Exterior Wall Decorations
    'Placards': 71,  # Placards
    'Fences': 72,  # Fences
    'Interior Walls': 73,  # Interior Walls
    'Floors': 74,  # Floors
    'Roof Lighting': 75,  # Roof Lighting
    'Garden Furnishings': 76,  # Garden Furnishings
    'Tables and Counters': 77,  # Tables and Counters
    'Tabletop': 78,  # Tabletop
    'Wall-mounted': 79,  # Wall-mounted
    'Carpets': 80,  # Carpets
    'Minions': 81,  # Pets
    'Gardening Items': 82,  # Gardening Items
    'Demimateria': 83,  # Demimateria
    'Dual Blades': 84,  # Equipment
    'Seasonal Miscellany': 85,  # Seasonal Miscellany
    'Orchestral Scrolls': 86,  # Materials
    'Two-handed Swords': 87,  # Equipment
    'Muskets': 88,  # Equipment
    'Astrolabes': 89,  # Equipment
    'Airship Components (Hull)': 90,  # Airship Components (Hull)
    'Airship Components (Rigging)': 91,  # Airship Components (Rigging)
    'Airship Components (Forecastle)': 92,  # Airship Components (Forecastle)
    'Airship Components (Aftcastle)': 93,  # Airship Components (Aftcastle)
    'Orchestrion Rolls': 94,  # Materials
    'Paintings': 95,  # Materials
    'Katana': 96,  # Equipment
    'Rapiers': 97,  # Equipment
    'Grimoires (SCH)': 98,  # Equipment
    'Fisher\'s Secondary Tool': 99,  # Equipment
    'Currency': 100,  # Currency
    'Submarine Components (Hull)': 101,  # Submarine Components (Hull)
    'Submarine Components (Tail)': 102,  # Submarine Components (Tail)
    'Submarine Components (Bow)' : 103,  # Submarine Components (Bow)
    'Submarine Components (Bridge)' : 104,  # Submarine Components (Bridge)
    'Blue Mage Arms' : 105,  # Blue Mage Arms
    'Gunblades' : 106,  # Equipment
    'Throwing Weapons' : 107,  # Equipment
    'Two-handed Scythes' : 108,  # Equipment
    'Trust Equipment' : 109,  # Equipment
}

In [5]:
path = os.path.join(root_dir, 'Items_v3.csv')

df = pd.read_csv(path, header=0)

df = df.dropna()

df = df.drop(df[df['IsUntradable'] == True].index)

categories = ['#', 'Name', 'Level{Item}', 'Rarity', 'FilterGroup', 'ItemUICategory',
       'ItemSearchCategory', 'EquipSlotCategory', 'ItemSortCategory',
       'StackSize', 'IsUnique', 'IsUntradable', 'Price{Mid}', 'Price{Low}',
       'CanBeHq', 'IsDyeable', 'ClassJob{Repair}', 'Level{Equip}',
       'EquipRestriction', 'ClassJobCategory', 'GrandCompany', 'ItemSeries',
       'MaterializeType', 'MateriaSlotCount', 'IsPvP', 'IsGlamourous']

#df.head(100)

#df.to_csv(os.path.join(root_dir, 'Items_v3.csv'), index=False)

In [6]:
url_dc = "https://universalis.app/api/v2/data-centers"
url_w = "https://universalis.app/api/v2/worlds"

response = requests.get(url_dc)

if response.status_code == 200:
    data_centers = response.json()
else:
    print("Error:", response.status_code)

response = requests.get(url_w)

if response.status_code == 200:
    worlds = response.json()
else:
    print("Error:", response.status_code)

target_dc = 5 # Chaos
target_worlds = data_centers[target_dc]['worlds']

In [7]:
for world in worlds:
  if world['id'] in target_worlds:
    print(world['name'], world['id'])

Omega 39
Moogle 71
Cerberus 80
Louisoix 83
Spriggan 85
Ragnarok 97
Sagittarius 400
Phantom 401


In [8]:
for world in target_worlds:
  url = f"https://universalis.app/api/v2/tax-rates?world={world}"
  response = requests.get(url)

  if response.status_code == 200:
      taxe = response.json()
      print(world, taxe)
  else:
      print("Error:", response.status_code)

39 {'Limsa Lominsa': 5, 'Gridania': 5, "Ul'dah": 5, 'Ishgard': 3, 'Kugane': 3, 'Crystarium': 3, 'Old Sharlayan': 3}
71 {'Limsa Lominsa': 5, 'Gridania': 5, "Ul'dah": 5, 'Ishgard': 3, 'Kugane': 3, 'Crystarium': 3, 'Old Sharlayan': 3}
80 {'Limsa Lominsa': 5, 'Gridania': 5, "Ul'dah": 5, 'Ishgard': 3, 'Kugane': 3, 'Crystarium': 3, 'Old Sharlayan': 3}
83 {'Limsa Lominsa': 5, 'Gridania': 5, "Ul'dah": 5, 'Ishgard': 3, 'Kugane': 3, 'Crystarium': 3, 'Old Sharlayan': 3}
85 {'Limsa Lominsa': 5, 'Gridania': 5, "Ul'dah": 5, 'Ishgard': 3, 'Kugane': 3, 'Crystarium': 3, 'Old Sharlayan': 3}
97 {'Limsa Lominsa': 5, 'Gridania': 5, "Ul'dah": 5, 'Ishgard': 3, 'Kugane': 3, 'Crystarium': 3, 'Old Sharlayan': 3}
400 {'Limsa Lominsa': 5, 'Gridania': 5, "Ul'dah": 5, 'Ishgard': 3, 'Kugane': 3, 'Crystarium': 0, 'Old Sharlayan': 0}
401 {'Limsa Lominsa': 5, 'Gridania': 5, "Ul'dah": 5, 'Ishgard': 3, 'Kugane': 3, 'Crystarium': 0, 'Old Sharlayan': 3}


In [23]:
def get_items(worldDcRegion, itemIds):
    url = f"https://universalis.app/api/v2/{worldDcRegion}/{itemIds}?listings=0&entries=0"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error:", response.status_code)
        return

In [21]:
fishs = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Fish']]['#'].to_list()
potions = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Potions']]['#'].to_list()
food = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Food']]['#'].to_list()

target_items = fishs # + potions + food

In [32]:
keys = [
    "itemID",
    "worldID",
    "lastUploadTime",
    "listings",
    "recentHistory",
    "currentAveragePrice",
    "currentAveragePriceNQ",
    "currentAveragePriceHQ",
    "regularSaleVelocity",
    "nqSaleVelocity",
    "hqSaleVelocity",
    "averagePrice",
    "averagePriceNQ",
    "averagePriceHQ",
    "minPrice",
    "minPriceNQ",
    "minPriceHQ",
    "",
    "maxPriceNQ",
    "maxPriceHQ",
    "stackSizeHistogram",
    "stackSizeHistogramNQ",
    "stackSizeHistogramHQ",
    "worldName",
]

In [34]:
item_keys = ['itemID', 'worldID', 'currentAveragePriceNQ', 'currentAveragePriceHQ']
df_items = pd.DataFrame(columns=item_keys)

for world in target_worlds:
    # separate the list of items into chunks of 100
    chunks = [target_items[x:x+100] for x in range(0, len(target_items), 100)]
    for chunk in chunks:
        data = get_items(world, chunk)
        if data:
            for item in data['items']:
                id = data['items'][item]['itemID']
                world = data['items'][item]['worldID']
                av_nq = data['items'][item]['currentAveragePriceNQ']
                av_hq = data['items'][item]['currentAveragePriceHQ']
                df_items = pd.concat([df_items, pd.DataFrame([[id, world, av_nq, av_hq]], columns=item_keys)])
        else:
            print("No data for", world)

In [36]:
df_items[df_items['itemID'] == 4869].head(100)

,itemID,worldID,currentAveragePriceNQ,currentAveragePriceHQ
0,4869,39,79479.500000,0
0,4869,71,487.307680,0
0,4869,80,66.000000,0
0,4869,83,409.190500,0
0,4869,85,401.882350,0
0,4869,97,135.818180,0
0,4869,400,123.611115,0
0,4869,401,206.312500,0


In [57]:
# compute the maximum differences between NQ average prices between worlds, and keep track of the two worlds

# Filter out rows with currentAveragePriceNQ equal to 0
df_comp = df_items[df_items['currentAveragePriceNQ'] != 0].copy()
df_comp = df_comp[df_comp['currentAveragePriceNQ'] < 10000]

# Compute the price difference for each itemID
df_comp['diff'] = df_comp.groupby('itemID')['currentAveragePriceNQ'].transform(lambda x: x.max() - x.min())

# Filter out rows with diff greater than 0 and sort in descending order
df_filtered = df_comp[df_comp['diff'] > 0].nlargest(100, 'diff')

df_diff = pd.DataFrame(columns=['itemID', 'worldID_max', 'worldID_min', 'diff'])

for item in df_filtered['itemID'].unique():
    # worldID_max is the world with the highest average price for the item
    worldID_max = df_filtered[df_filtered['itemID'] == item].nlargest(1, 'currentAveragePriceNQ')['worldID'].values[0]
    # worldID_min is the world with the lowest average price for the item
    worldID_min = df_filtered[df_filtered['itemID'] == item].nsmallest(1, 'currentAveragePriceNQ')['worldID'].values[0]
    # diff is the difference between the highest and lowest average price for the item
    diff = df_filtered[df_filtered['itemID'] == item]['diff'].values[0]
    df_diff = pd.concat([df_diff, pd.DataFrame([[item, worldID_max, worldID_min, diff]], columns=['itemID', 'worldID_max', 'worldID_min', 'diff'])])

# replace worldID with worldName and itemID with itemName
df_diff['worldName_max'] = df_diff['worldID_max'].map(lambda x: next((item for item in worlds if item['id'] == x), None)['name'])
df_diff['worldName_min'] = df_diff['worldID_min'].map(lambda x: next((item for item in worlds if item['id'] == x), None)['name'])
df_diff['itemName'] = df_diff['itemID'].map(lambda x: next((item for item in df[['#', 'Name']].values if item[0] == x), None)[1])

# rmove worldID_max, worldID_min and itemID columns
df_diff = df_diff.drop(columns=['worldID_max', 'worldID_min', 'itemID'])

# reorder columns
df_diff = df_diff[['itemName', 'worldName_max', 'worldName_min', 'diff']]

# remove index
df_diff = df_diff.reset_index(drop=True)

df_diff.head(100)

,itemName,worldName_max,worldName_min,diff
0,Thousandfang,Sagittarius,Moogle,9870.50000
1,Mummer Wrasse,Phantom,Spriggan,9859.00000
2,Nogoi,Sagittarius,Cerberus,9731.50000
3,Sea Bo,Cerberus,Phantom,9730.00000
4,Herring Pie,Sagittarius,Moogle,9644.00000
5,Aetherlouse,Sagittarius,Louisoix,9627.83300
6,Pirate's Bane,Phantom,Omega,9625.45000
7,Soul of the Stallion,Cerberus,Ragnarok,9576.33334
8,Crowned Pie,Cerberus,Ragnarok,9548.01100
9,Triple Cream Coffee,Moogle,Phantom,9505.75554
